In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mgimg
import matplotlib.colors as colors
import scipy as sp
import numpy as np
import emcee
import testkit
import corner
import pickle as pickle
from IPython.display import display
%matplotlib inline
import forwardmodel
import ciamod
import TPmod
import cloud
import band
import brewtools
from astropy.convolution import convolve, convolve_fft
from astropy.convolution import Gaussian1DKernel
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.interpolate import InterpolatedUnivariateSpline
from bensconv import prism_non_uniform
from bensconv import conv_uniform_R
from bensconv import conv_uniform_FWHM

## First step is to load the results file and the run arguments. 
We open the files and select the last couple of thousand iterations of the chain, and flatted this into a simple array of state vectors called "samples"

In [ ]:
path = "./"
runname = "WISE1935_NC_vsini"
# error for distance and photometry for corner plot R and M
sigDist = 0.8
sigPhot = 0.02 
# OK finish? 1 for yes, 0 for no.
fin = 1
flatendchain, flatendprobs,ndim = brewtools.get_endchain(runname,fin)
theta_max_end = flatendchain[np.argmax(flatendprobs)]
max_end_like = np.amax(flatendprobs)
samples = flatendchain

argfile =runname+"_runargs.pic"

runargs = brewtools.pickle_load(argfile)
# If you're opening on a Linux box, use the code below
#with open(argfile, 'rb') as input:
#    runargs = pickle.load(input) 

gases_myP,chemeq,dist, cloudtype,do_clouds,gasnum,cloudnum,inlinetemps,coarsePress,press,inwavenum,linelist,cia,ciatemps,use_disort,fwhm,obspec,proftype,do_fudge, prof,do_bff,bff_raw,ceTgrid,metscale,coscale = runargs



Then we print the max likelihood value for the state vector, just to have a look at it

In [ ]:
print(theta_max_end)

And print the BIC for this run. You might want to take a note of this.

In [ ]:
BIC = (-2.* max_end_like) + (ndim*np.log(obspec.shape[1]))
print("BIC = "+str(BIC))

Next we want to plot the profile. We start by calculating the profiles from the temperature parameters in the samples array. 

This case assumes a type 1 (Line et al 2015), 13 knot profile. 

In [ ]:
Tsamples = samples[:,ndim-13:]
nsamps = Tsamples.shape[0]
Tprofs = np.empty([64,Tsamples.shape[0]])
for i in range(0,nsamps):
    Tprofs[:,i] = TPmod.set_prof(1,coarsePress,press,Tsamples[i,:])

Tlays = np.empty([64,5])
for i in range(0,64):
    junk = Tprofs[i,:]
    junk2 = np.percentile(junk, [2.4,16, 50, 84,97.6],axis=0)
    junk3 = np.array(junk2)
    Tlays[i,:] = junk3[:]
    

In [ ]:
# max likelihood Tprof
bestT = TPmod.set_prof(1,coarsePress,press,theta_max_end[ndim-13:])

Now we can plot it... 

In [ ]:
modP1,modT1 = np.loadtxt("/Volumes/DudleyDisk/SONORA/Bobcat2021/Structures/t400g1000nc_m+0.5.dat",skiprows=1,usecols=(1,2),unpack=True)
modP2,modT2 = np.loadtxt("/Volumes/DudleyDisk/SONORA/Bobcat2021/Structures/t350g316nc_m+0.5.dat",skiprows=1,usecols=(1,2),unpack=True)
modP3,modT3 = np.loadtxt("/Volumes/DudleyDisk/SONORA/Bobcat2021/Structures/t500g3160nc_m+0.5.dat",skiprows=1,usecols=(1,2),unpack=True)


In [ ]:
plt.rc('font',family='Times New Roman')
plt.rc('text', usetex=True)

fig=plt.figure(dpi=320)
plt.axis([0., 1500.,3.0,-4.0])

logP = np.log10(press)

d1, = plt.plot(Tlays[:,2],logP,'k-')
plt.fill_betweenx(logP,Tlays[:,1], Tlays[:,3], facecolor='red', alpha=0.3)
plt.fill_betweenx(logP,Tlays[:,0], Tlays[:,4], facecolor='red', alpha=0.1)

l1, = plt.plot(bestT,logP,'g--')

m1, = plt.plot(modT1,np.log10(modP1),'m-',linewidth=1,label='Bobcat 400K logg = 5.0 [M/H]=+0.3')
m2, = plt.plot(modT2,np.log10(modP2),'c-',linewidth=1,label='Bobcat 350K logg = 4.5 [M/H]=+0.3')
m3, = plt.plot(modT3,np.log10(modP3),'y-',linewidth=1,label='Bobcat 500K logg = 5.5 [M/H]=+0.3')

# Here are some condensation curves
mns = 10.0**4/(7.45 - 0.42*logP-0.84*0.0)
na2s = 10.0**4/(10.05 - 0.72*logP-1.08*0.0)
zns = 10.0**4/(12.52 - 0.63*logP-1.26*0.0)
kcl = 10.0**4/(12.48 - 0.8786*logP-0.8786*0.0)
nh4h2po4 = 10.0**4/(29.99-0.20*(11.0*logP + 15.0*0.0))
h2o = 10000.0 / (40.042-3.730*logP - 3.730*0.0)

c1, = plt.plot(mns,logP,'--',color='yellow',linewidth=1.5, label='MnS')
c2, = plt.plot(na2s,logP,'--',color='pink',linewidth=1.5,label='Na2S')
c3, = plt.plot(zns,logP,'--',color='orange',linewidth=1.5, label='ZnS')
c4, = plt.plot(kcl,logP,'--',color='purple',linewidth=1.5, label='KCl')
#c5, = plt.plot(nh4h2po4,logP,'--',color='red',linewidth=1.5, label='NH$_4$H$_2$PO$_4$')
c6, = plt.plot(h2o,logP,'--',color='blue',linewidth=1.5, label='H2O')

#enst = 10.0**4/(6.26 - 0.35*logP-0.70*0.0)
#fost = 10.0**4/(5.89 - 0.37*logP-0.73*0.0)
#iron = 10.0**4/(5.44 - 0.48*logP-0.48*0.0)
#cr =  10.0**4/(6.528 - 0.491*logP-0.491*0.0)
#al2o3 = 10.0**4 / (5.0139 - 0.21794*(logP) + 2.2636E-03*(logP)**2.0 - 0.580*0.0)
#c1, = plt.plot(enst,logP,'--',color='blue',linewidth=1.5, label='MgSiO$_3$')
#c2, = plt.plot(fost,logP,'--',color='pink',linewidth=1.5,label='Mg$_2$SiO$_4$')
#c3, = plt.plot(iron,logP,'--',color='orange',linewidth=1.5, label='Fe')
#c4, = plt.plot(cr,logP,'--',color='purple',linewidth=1.5, label='Cr')
#c5, = plt.plot(al2o3,logP,'--',color='red',linewidth=1.5, label='Al$_2$O$_3$')

plt.legend(handles=[d1,m3,m1,m2,c1,c2,c3,c4,c6],loc=1)
plt.ylabel(r'log(P / bar)')
plt.xlabel('T / K')

#asp = 10 / 3.5

#plt.axes().set_aspect(asp)
#plt.savefig(runname+"_profile.png",format='png', dpi=320)

In [ ]:
# Save the median profile
np.savetxt(runname+'_profile.dat', np.c_[logP, Tlays[:,2]])

# Next we want to plot the gas abundances in a corner plot.

In [ ]:
# You'll need to edit this cell to make it work for the gases you've used
# e.g. change the cut you take out of the samples array to get gases and gravity
gassamples = samples[:,0:14].copy()
# set up boundaries for histograms
rh2o = (-3.0,-2.5)
rch4 = (-3.1,-2.5)
rco = (-5.4,-4.9)
rco2 = (-8, -7.45)
rnh3 = (-4.3,-3.8)
rh2s = (-4.5,-4.0)
rph3 = (-12,0)
rmet = [0.,1.0]
rco_rat = [0.5,1.0]
rlogg = (4.7,5.2)
rRad = (0.9,1.5)
rMass = (25,100)
rvrad = (-45,-35)
rvsini = (0,30)


# get M/H and C/O
h2o = gassamples[:,0]
ch4 = gassamples[:,1]
co = gassamples[:,2]
co2 = gassamples[:,3]
nh3 = gassamples[:,4]
h2s = gassamples[:,5]
ph3 = gassamples[:,6]

# first get the C/O

O = 10**h2o + 10**co + 2.*10**(co2) 
C = 10**(co) + 10**(co2) + 10**(ch4)

CO_ratio = C/O

# won't bother with rest of the elements as they're not fully accounted for
# i.e. N is hiding in N2, S maybe hiding elsewhere, 
# and we haven't detected PH3
    

# Determine "fraction" of H2 in the L dwarf
gas_sum = 10**h2o + 10**co +10**co2 + 10**ch4
fH2 = (1-gas_sum)* 0.84  # fH2/(fH2+FHe) = 0.84
fH = 2.*fH2

    
# Determine linear solar abundance sum of elements in our L dwarf
# abundances taken from Asplund+ 2009
solar_H = 12.00
solar_O = 10**(8.69-solar_H)
solar_C = 10**(8.43-solar_H) 
#solar_Ti = 10**(4.95-solar_H)
#solar_V = 10**(3.93-solar_H)
#solar_Cr = 10**(5.64-solar_H)
#solar_Fe = 10**(7.50-solar_H)
#solar_NaK = 10**(6.24-solar_H) + 10**(5.03-solar_H)
    
# Calculate the metallicity fraction in the star and the same for the sun and then make the ratio
metallicity_target = (O/fH) + (C/fH) 
metallicity_sun = solar_O + solar_C 

MH = np.log10(metallicity_target / metallicity_sun)

# Get the radius and mass
r2d2 = samples[:,8].copy()
logg = samples[:,7].copy()
# Radius
sigR2D2 = sigPhot * r2d2 * (-1./2.5)* np.log(10.)

sigD = sigDist * 3.086e16
D = dist * 3.086e16

R = np.sqrt(((np.random.randn(len(samples[:,0])) * sigR2D2)+ r2d2)) * ((np.random.randn(len(samples[:,0]))* sigD) + D)


# and mass
g = (10.**logg)/100.
M = (R**2 * g/(6.67E-11))/1.898E27
 
R = R / 71492e3

gassamples[:,7] = MH
gassamples[:,8] = CO_ratio
gassamples[:,9] = logg
gassamples[:,10] = R
gassamples[:,11] = M
gassamples[:,12:14] = samples[:,9:11].copy()
# now make an array of the bounds to give to corner plot
bnds = [rh2o,rch4,rco,rco2,rnh3,rh2s,rph3,rmet,rco_rat,rlogg,rRad,rMass,rvrad,rvsini]


fig = corner.corner(gassamples,scale_hist=False, plot_datapoints =False,labels=["H$_2$O","CH$_4$","CO","CO$_2$","NH$_3$","H$_2$S","PH$_3$","[M/H]","C/O","$\log g$","$R / R_{Jup}$","$M / M_{Jup}$","$v_{rad}$","$v \sin i$"],quantiles=[0.16, 0.5, 0.84],show_titles=True, title_kwargs={"fontsize": 20,"loc":"left"},label_kwargs={"fontsize": 20})
#plt.savefig(runname+"_MRgascorner.png",bbox_inches='tight',format='png', dpi=320)


In [ ]:
# This saves out the gas mixing ratios
gasVMR = np.ones([gasnum.size,3])
for i in range (0,gasnum.size):
    gasVMR[i,:] = np.percentile(samples[:,i], [16, 50, 84])
    
np.savetxt(runname+'_VMRs.dat', np.c_[gasVMR[:,0],gasVMR[:,1],gasVMR[:,2]])


# Plotting the model spectra
To plot the model spectra we need to rerun the model, and rerun it for a bunch of random draws from the posterior. This will allow us to plot either a spaghetti plot, or a median + interval spectrum

In [ ]:
# no need to get diagnostics along with the spectrum
gnostics = 0
# Now run the model again to get your model spectrum and process to make it look like the data
shiftspec, photspec, tauspec,cfunc = testkit.modelspec(theta_max_end,runargs,gnostics)
topspec = brewtools.proc_spec(shiftspec,theta_max_end,fwhm,chemeq,gasnum,obspec) 


In [ ]:
print(topspec)

In [ ]:
print(theta_max_end)

In [ ]:
# Now grab 500 random draws from the posterior
pltspec = np.zeros((100,obspec[0,:].size))
samp= np.empty(ndim)
sid = np.zeros(100)
for i in range (0,100):
    sid[i]= np.random.randint(0,high = len(samples))
    samp = samples[int(sid[i]),:]
    shiftspec, photspec, tauspec,cfunc = testkit.modelspec(samp,runargs,gnostics)
    pltspec[i,:] = brewtools.proc_spec(shiftspec,samp,fwhm,chemeq,gasnum,obspec) 
    
# get the intervals for the distribution of model spectra
specdist = np.empty([obspec[0].size,5])
for i in range(0,obspec[0].size):
    junk = pltspec[:,i]
    junk2 = np.percentile(junk[~np.isnan(junk)], [2.4,16, 50, 84,97.6])
    junk3 = np.array(junk2)
    specdist[i,:] = junk3[:]

# plot the spectra

In [ ]:
plt.rc('font',family='Times New Roman')
plt.rc('text', usetex=True)
fig=plt.figure(dpi=320)
fig, axs = plt.subplots(6,gridspec_kw={'hspace': 1, 'wspace': 0})

asp = 10.0

for i in range(0,6):
    lsize = 8

    axs[i].set(xlim=[2.8+(i*0.4),3.2+(i*0.4)])
    # set the plot range for each plot
    xr =  np.where(np.logical_and(obspec[0,:] > (2.8+(i*0.4)),obspec[0,:] < 3.2+(i*0.4)))
    xr = xr[0]
    
    d1, = axs[i].plot(obspec[0,xr],obspec[1,xr]/1e-17,'k-',label = runname+" data")
    axs[i].fill_between(obspec[0,xr],(obspec[1,xr]-obspec[2,xr])/1e-17,(obspec[1,xr]+obspec[2,xr])/1e-17,facecolor='gray',alpha=0.5)
    r1, = axs[i].plot(obspec[0,xr],specdist[xr,2]/1e-17,'y-',linewidth=0.8, label = "median")
    plt.fill_between(obspec[0],specdist[:,0],specdist[:,4],facecolor='red',alpha=0.2)
    axs[i].fill_between(obspec[0,xr],specdist[xr,1]/1e-17,specdist[xr,3]/1e-17,facecolor='red',alpha=0.5)

    t1, = axs[i].plot(obspec[0,xr],topspec[xr]/1e-17,'c-',linewidth=0.5, label = "max likelihood")
    
    if i == 0:
        axs[i].legend(handles=[d1,t1],bbox_to_anchor=(1.05,1.2),loc='upper left')
    
    
#plt.ylabel(r'$ F_{\lambda}$ / $10^{-17} Wm^{-2} \mu m^{-1}$',loc='center')
#plt.xlabel('Wavelength / $\mu m$')


fig.txt = fig.text(0.5, 0.04,'Wavelength / $\mu m$', ha='center')
fig.text(0.05, 0.5,r'$ F_{\lambda}$ / $10^{-17}~{\rm Wm^{-2} \mu m^{-1}}$' , va='center', rotation='vertical')


#plt.savefig(runname+"_panel_spec.png",format='png', dpi=320)

In [ ]:
stat= np.sum(((obspec[1,:] - topspec)**2) / (obspec[2,:]**2 + 10**theta_max_end[ndim-15]))
    # And divide it by the number of data points to get reduced chi^2
chi2 = stat / len(obspec[0])
print('with tolerance parameter chi2 = ',chi2)
stat= np.sum(((obspec[1,:] - topspec)**2) / (obspec[2,:]**2))
    # And divide it by the number of data points to get reduced chi^2
chi2 = stat / len(obspec[0])

print('without tolerance parameter chi2 = ',chi2)

In [ ]:
# Save you  model spectra
np.savetxt(runname+'_MEDIAN_SPEC.dat', np.c_[obspec[0,:],specdist[:,2],specdist[:,1],specdist[:,3]])
np.savetxt(runname+'_MAX_LIKE_SPEC.dat', np.c_[obspec[0,:],topspec])

# Contribution function
Finally, the contribution function. This is a really useful plot...

In [ ]:
# get diagnostics along with the spectrum
gnostics = 1
shiftspec, clphotspec, ophotspec,cfunc = testkit.modelspec(theta_max_end,runargs,gnostics)

nwave = inwavenum.size
cfunc = np.reshape(cfunc,[cfunc.shape[1],cfunc.shape[2]])
fwhm = 4/3000
wlen = shiftspec.shape[1]
wint =  shiftspec[0,0] - shiftspec[0,wlen-1]
# convolve with instrumental profile
# start by setting up kernel
# First step is finding the array index length of the FWHM
disp = wint / wlen
gwidth = int((((fwhm / disp) // 2) * 2) +1)
# needs to be odd
# now get the kernel and convolve
gauss = Gaussian1DKernel(gwidth)

for ilayer in range (0,press.size):
    cfunc[:,ilayer] = convolve(cfunc[:,ilayer],gauss,boundary='extend')

tau1_cl_Press = convolve(clphotspec[0],gauss,boundary='extend')[::-1]
tau1_oth_Press = convolve(ophotspec[0],gauss,boundary='extend')[::-1]
    
wavenew = shiftspec[0,::-1]
press = press.reshape(64,)
normfunc = np.zeros_like(cfunc)
for iwave in range(0,nwave):
    totcont = np.sum(cfunc[iwave,:])
    normfunc[iwave,:] = cfunc[iwave,:] / totcont

    
plt.rc('font', family='serif')
plt.rc('text', usetex=False)
#fig=plt.figure(dpi=120)

fig, ax = plt.subplots()
ax.axis([wavenew[0],wavenew[-1],press[-1],press[0]])

ax.set_yscale('log')
ax.set_xscale('log')
#major_ticks = np.arange(1.0,15.,1.0)
#minor_ticks = np.arange(1.0,15.,0.5)
#ax.set_xticks(major_ticks)                                                       
#ax.set_xticks(minor_ticks, minor=True)                                           

cm = ax.pcolormesh(wavenew,press,(normfunc[::-1,:].transpose()),cmap='Greys',norm=colors.SymLogNorm(linthresh=0.001,linscale=0.00001,
                                              vmin=0., vmax=np.amax(normfunc)))

#t1, = plt.plot(wavenew,(tau1_cl_Press),'m-',label=r'$\tau_{cloud} = 1.0$')
#t2, = ax.plot(wavenew,(tau1_oth_Press),'c-', label =r'$\tau_{gas} = 1.0$')

#fig.legend(handles=[t2])

cbar = fig.colorbar(cm,ax=ax,orientation='vertical',norm=colors.Normalize(clip=False),ticks=[1e-3,1e-2,0.1])
cbar.ax.set_yticklabels(['<0.1%', '1%', '10%'])
cbar.set_label('% of total', rotation=270)
plt.ylabel('(Pressure / bar)')
plt.xlabel('Wavelength / $\mu m$')
plt.savefig(runname+'_contribution.png',format='png', dpi=120)

In [ ]:
# grab BFF and Chemical grids
gaslist = ['h2o','ch4','co','co2','nh3','h2s','ph3']
print(gaslist)
chemeq=1
bff_raw,ceTgrid,metscale,coscale,gases_myP = testkit.sort_bff_and_CE(chemeq,"chem_eq_tables_P3K.pic",press,gaslist)
   
# what metallicity and C/O ratio do we want to plot?
mh  = 0.3
co = (0.55/0.55)

logP, profT = np.loadtxt(runname+"_profile.dat",unpack=True)

nlayers = press.size
mfit = interp1d(metscale,gases_myP,axis=0)
gases_myM = mfit(mh)
cfit = interp1d(coscale,gases_myM,axis=0)
invmr = cfit(co)
ng = invmr.shape[2]
ngas = len(gaslist)
ab = np.zeros([nlayers,ngas],dtype='d')
temp= profT #Tlays[:,2]
for p in range(0,nlayers):
    for g in range(3,ng):
        tfit = InterpolatedUnivariateSpline(ceTgrid,invmr[:,p,g])
        ab[p,g-3]= tfit(temp[p])

vmr = gasVMR

plt.rc('font',family='Times New Roman')
plt.rc('text', usetex=True)
fig=plt.figure(dpi=320)
fig.set_size_inches(6., 6.)
fig, axs = plt.subplots(ngas,sharex='col', sharey='row',gridspec_kw={'hspace': 0, 'wspace': 0})


asp = 0.5

for i in range(0,ngas):
    lsize = 8
    # row and column sharing

    axs[i].set(xlim=[-12, 0], ylim=[2.5,-4], aspect=asp)
    axs[i].plot(ab[:,i],logP,'--',color='red',linewidth=1.5)
    c1, = axs[i].plot([vmr[i,1],vmr[i,1]],[2.4,-4.0],'-',color='red',label=gaslist[i].upper())
    axs[i].fill_betweenx([2.4,-4.0],[vmr[i,0],vmr[i,0]],[vmr[i,2],vmr[i,2]], facecolor='red', alpha=0.3,linewidth=0)
    axs[i].legend(handles=[c1],prop={'size':lsize})

# this bit is Na+K tied together. If they're not, remove this bit, and run the loop above to ngas
#alks = np.log10((10.**ab[:,ngas-2]) + (10.**ab[:,ngas-1]))

#axs[ngas-2].set(xlim=[-12, 0], ylim=[2.5,-4], aspect=asp)  
#axs[ngas-2].tick_params(axis='y',direction='in',left=True,right=True)
#axs[ngas-2].plot(alks,logP,'--',color='red',linewidth=2)
#c7, = axs[ngas-2].plot([vmr[ngas-2,1],vmr[ngas-2,1]],[2.4,-4.0],'-',color='red',label='Na+K')
#axs[ngas-2].fill_betweenx([2.4,-4.0],[vmr[ngas-2,0],vmr[ngas-2,0]],[vmr[ngas-2,2],vmr[ngas-2,2]], facecolor='red', alpha=0.1,linewidth=0)
#axs[ngas-2].legend(handles=[c7],prop={'size':lsize})



fig.txt = fig.text(0.5, 0.04, '$\log f_{gas}$', ha='center')
fig.text(0.3, 0.5, '$\log (P / bar)$', va='center', rotation='vertical')
plt.savefig(runname+'_abundances.png',format='png', dpi=320)